In [1]:
import os
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Coron3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from glob import glob
import astropy.io.fits as fits
import numpy as np
import time
import pandas as pd
import json
import pickle
from json import JSONEncoder
from collections import defaultdict
from CoronPipeline import MyCoron3Pipeline

In [2]:
os.environ["CRDS_PATH"] = '/home/sarperyn/crds_cache/jwst_ops'
os.environ["CRDS_SERVER_URL"] = 'https://jwst-crds.stsci.edu'

In [3]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

# figure sizes and resolutions
rcParams['figure.figsize'] = [20,15]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

In [4]:
import jwst
print(jwst.__version__)

1.10.2


In [5]:
PROPOSAL_ID = '1386'
INSTRUME = 'NIRCAM'
#DETECTOR = 'nrca2'

In [6]:
import os
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Coron3Pipeline
import jwst.associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from glob import glob
import astropy.io.fits as fits
import numpy as np
import time
import pandas as pd
import json
import pickle
from json import JSONEncoder
from collections import defaultdict
from CoronPipeline import MyCoron3Pipeline
from jwst.associations.mkpool import mkpool
from jwst.associations.lib.rules_level3 import Asn_Lv3Coron
from jwst.associations import AssociationPool, AssociationRegistry
from jwst.associations.mkpool import from_cmdline, mkpool
from jwst.associations import generate
from astropy.table import Table

In [7]:
def get_exptypes(fits_):
    
    for file in range(len(fits_)):
        
        f = fits.open(fits_[file])
        exp_type = f[0].header['EXP_TYPE']
        print(exp_type)
        
def runimg2(filename, output_dir):
    
    img2 = Image2Pipeline()
    img2.output_dir = output_dir
    img2.save_results = True
    img2(filename)

def t_path(partial_path):
    __file__ = '/home/sarperyn/sarperyurtseven/ProjectFiles/notebooks/asn_coron_rule.py'
    test_dir = os.path.dirname(__file__)
    return os.path.join(test_dir, partial_path)


def runcoron(filename, output_dir):

    coron = MyCoron3Pipeline()
    coron.output_dir = output_dir
    coron.save_results = True
    coron.process(filename)

def get_stage3_products(asns,directory):
    
    for t in range(len(asns)):

        asn_dict = {}
        for i,j in zip(asns[t].keys(),asns[t].values()):
            asn_dict[i] = j
            
        runcoron(asn_dict,directory)
        

In [8]:
def process_products(programs):

    for program in programs:
        
        directory = f'/data/scratch/bariskurtkaya/dataset/{INSTRUME}/{program}/mastDownload/JWST/'
        rateints_files = glob(os.path.join(directory, '*/*rateints.fits'))
        batch_size = 4
        for i in range(0,len(rateints_files),batch_size):
        
            for f in rateints_files[i:i+batch_size]:
                
                output_dir = '/'.join(f.split('/')[:-1]) + '/' 
                runimg2(f,output_dir)
            time.sleep(1)
    
        calints_data = glob(os.path.join(directory, '**/**calints.fits'))
    
        pool = mkpool(calints_data)
        pool_df = pool.to_pandas()
        pool_df.to_csv(f'calints_{INSTRUME}_{program}_pool.csv',index=False)
    
        registry = AssociationRegistry([t_path('rule_level3.py')], include_default=False)
        asns = generate(pool,registry)
    
        get_stage3_products(asns,directory)

In [9]:
programs = ['1386']
process_products(programs)

2023-07-04 07:07:20,294 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2023-07-04 07:07:20,297 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2023-07-04 07:07:20,301 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2023-07-04 07:07:20,303 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2023-07-04 07:07:20,307 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2023-07-04 07:07:20,311 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2023-07-04 07:07:20,459 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386001001_03102_00001_nrcalong/jw01386001001_03102_00001_nrcalong_rateints.fits',).
2023-07-04 07:07:20,466 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/

2023-07-04 07:07:21,397 - stpipe.Image2Pipeline - INFO - Prefetching reference files for dataset: 'jw01386001001_03102_00001_nrcalong_rateints.fits' reftypes = ['area', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'photom', 'regions', 'sflat', 'specwcs', 'wavelengthrange', 'wfssbkg']
2023-07-04 07:07:22,030 - stpipe.Image2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2023-07-04 07:07:22,031 - stpipe.Image2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2023-07-04 07:07:22,032 - stpipe.Image2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2023-07-04 07:07:22,032 - stpipe.Image2Pipeline - INFO - Prefetch for DFLAT reference file is 'N/A'.
2023-07-04 07:07:22,033 - stpipe.Image2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2023-07-04 07:07:22,034 - stpipe.Image2Pipeline - INFO - Prefetch for DISTORT

psf_sub didnt work


IndexError: list index out of range

# Pipeline things



<div class="alert alert-block alert-info">
Now we have the fits files. We will create our coronagrahic images using the JWST pipeline. Our data has the suffix <b>.rateints</b> just like we needed. 
While processing coronagraphy data, the common procedure is using the following pipeline:
    <ul> 
        <li> calwebb_detector1 $\rightarrow$ <strong>from jwst.pipeline import Detector1Pipeline</strong>
        <li> calwebb_image2  $\rightarrow$ <strong>from jwst.pipeline import Image2Pipeline</strong>
        <li> calwebb_coron3  $\rightarrow$ <strong>from jwst.pipeline import Coron3Pipeline</strong>
    </ul>
</div>

We have the outputs of the first stage. They all have the `rateints.fits` suffix. By passing them through the second pipeline we are going to end up with `calints.fits` files which is a last step before getting the coron products.

In [14]:
# Path containing the rateints data
directory = f'/data/scratch/bariskurtkaya/dataset/{INSTRUME}/{PROPOSAL_ID}/mastDownload/JWST/'

### Stage2

In [15]:
def get_exptypes(fits_):
    
    for file in range(len(fits_)):
        
        f = fits.open(fits_[file])
        exp_type = f[0].header['EXP_TYPE']
        print(exp_type)

In [16]:
def runimage(filename, output_dir):
    
    img2 = Image2Pipeline()
    img2.output_dir = output_dir
    img2.save_results = True
    img2(filename)

In [17]:
def run_pipeline2(directory):
    rateints_files = glob(os.path.join(directory, '*/*rateints.fits'))
    print('rateints file:',len(rateints_files))
    batch_size = 4
    for i in range(0,len(rateints_files),batch_size):
    
        for f in rateints_files[i:i+batch_size]:
            
            output_dir = '/'.join(f.split('/')[:-1]) + '/' 
            runimg2(f,output_dir)
        time.sleep(1)

In [18]:
calints_data = glob(os.path.join(directory, '*/*calints.fits'))

In [19]:
directory

'/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/'

In [20]:
len(calints_data)

101

### Stage 3

The stage 3 is kinda different than the other steps. To run this step we need an association (ASN) file. Thus, we need some functions to create ASN files.

In [21]:
from jwst.associations.mkpool import mkpool
from jwst.associations.lib.rules_level3 import Asn_Lv3Coron
from jwst.associations import AssociationPool, AssociationRegistry
from jwst.associations.mkpool import from_cmdline, mkpool
from jwst.associations import generate
from astropy.table import Table

In [26]:
def get_asns_files(calints_data,INSTRUME,PROPOSAL_ID):

    pool = mkpool(calints_data)
    pool_df = pool.to_pandas()
    pool_df.to_csv(f'calints_{INSTRUME}_{PROPOSAL_ID}_pool.csv',index=False)
    registry = AssociationRegistry([t_path('rule_level3.py')], include_default=False)
    asns = generate(pool,registry)
    print(f'Length of association pools for the program {PROPOSAL_ID}:',len(asns))

    return asns



In [23]:
def t_path(partial_path):
    __file__ = '/home/sarperyn/sarperyurtseven/ProjectFiles/notebooks/asn_coron_rule.py'
    test_dir = os.path.dirname(__file__)
    return os.path.join(test_dir, partial_path)

In [24]:
def runcoron(asn_dict, output_dir):

    coron = MyCoron3Pipeline()
    coron.output_dir = output_dir
    coron.save_results = True
    coron.process(asn_dict)

In [25]:
def run_pipeline3(asns,directory):
    
    for t in range(len(asns)):

        asn_dict = {}
        for i,j in zip(asns[t].keys(),asns[t].values()):
            asn_dict[i] = j
            
        runcoron(asn_dict,directory)
        
        

In [27]:
asns = get_asns_files(calints_data,INSTRUME,PROPOSAL_ID)

202307040732:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386001001_03106_00003_nrcalong/jw01386001001_03106_00003_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202307040732:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386002001_03109_00001_nrcalong/jw01386002001_03109_00001_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202307040732:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/mastDownload/JWST/jw01386001001_0310e_00003_nrcalong/jw01386001001_0310e_00003_nrcalong_calints.fits" is not a Level 1b name. Cannot transform to Level 2b.
202307040732:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="/data/scratch/ba

Length of association pools for the program 1386: 7


In [29]:
run_pipeline3(asns,directory)

2023-07-04 07:33:50,241 - stpipe.MyCoron3Pipeline - INFO - MyCoron3Pipeline instance created.
2023-07-04 07:33:50,244 - stpipe.MyCoron3Pipeline.stack_refs - INFO - StackRefsStep instance created.
2023-07-04 07:33:50,247 - stpipe.MyCoron3Pipeline.align_refs - INFO - AlignRefsStep instance created.
2023-07-04 07:33:50,249 - stpipe.MyCoron3Pipeline.klip - INFO - KlipStep instance created.
2023-07-04 07:33:50,253 - stpipe.MyCoron3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2023-07-04 07:33:50,256 - stpipe.MyCoron3Pipeline.resample - INFO - ResampleStep instance created.
2023-07-04 07:33:50,258 - stpipe.MyCoron3Pipeline - INFO - Starting calwebb_coron3 ...
2023-07-04 07:33:53,299 - stpipe.MyCoron3Pipeline - INFO - Prefetching reference files for dataset: 'jw01386001001_0310c_00004_nrcalong_calints.fits' reftypes = ['drizpars', 'psfmask']
2023-07-04 07:33:53,808 - stpipe.MyCoron3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/home/sarperyn/crds_cach

psf_sub didnt work


IndexError: list index out of range